In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import glob
import shutil

In [0]:
#shutil.copy("drive/My Drive/weather_turkey_processed.zip","./")
#!unzip weather_turkey_processed.zip

In [0]:
shutil.copy("drive/My Drive/RealTimeConsumption-01122014-09032020.csv","./")

'./RealTimeConsumption-01122014-09032020.csv'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

In [134]:
consumption_df=pd.read_csv("RealTimeConsumption-01122014-09032020.csv",encoding='latin-1')
consumption_df.columns=["Date","Hour","Consumption"]

consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
consumption_df["Consumption"]=consumption_df["Consumption"].apply(lambda x:x.replace(",","")).astype(float)
consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"],format="%d.%m.%Y %H:%M")
consumption_df=consumption_df.loc[consumption_df["Datetime"]<pd.to_datetime("2020-03-04 00:00:00")]
consumption_df#.loc[consumption_df["Date"]=="06.03.2019"]

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00


In [0]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
exp_eval_df=[]
def eval_function(name_,y_true, y_pred):
    print("RMSE: ",np.sqrt(mean_squared_error(y_true,y_pred)))
    #print("RMSLE: ",np.sqrt(mean_squared_log_error(y_true,y_pred)))
    print("MAE: ",mean_absolute_error(y_true,y_pred))
    print("MAPE: ",np.mean(np.abs((y_true - y_pred) / y_true)) * 100 )
    return name_, np.sqrt(mean_squared_error(y_true,y_pred)),mean_absolute_error(y_true,y_pred), np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#HW Exponential Smoothing without Special days and weekly Cycles

In [0]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [0]:
ExpSmt = ExponentialSmoothing(consumption_df["Consumption"], seasonal_periods=24, trend='add', seasonal='add').fit(use_boxcox=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


In [0]:
exp_eval_df.append(eval_function("HW_ExpoSth",consumption_df["Consumption"],ExpSmt.fittedvalues))

RMSE:  646.3209311253985
MAE:  447.9737093609095
MAPE:  1.412794376192841


#HW Exponential Smoothing for Each Hour Including Weekly Cycles

In [137]:
predicted_values=[[]]
pred_list=[]
#consumption_df['HW_predictions']=5
for hour in np.unique(consumption_df["Hour"]):
  Exp=ExponentialSmoothing(consumption_df.loc[consumption_df['Hour'] == hour]["Consumption"], seasonal_periods=7, trend='add', seasonal='add').fit(use_boxcox=True)
  #df=pd.DataFrame(Exp.fittedvalues)
  predicted_values.append(Exp.fittedvalues)
  #pred_df=pd.DataFrame(list(zip(consumption_df.loc[consumption_df['Hour'] == hour]["Consumption"],Exp.fittedvalues)), columns =['real', 'pred']) 
  #consumption_df.loc[consumption_df['Hour'] == hour]['HW_predictions']=5
  #Exp.fittedvalues
  #MAPEs.append(eval_function("HW_ExpoSth",pred_df["real"],pred_df["pred"])[3])
#cons_lagged_exp=cons_lagged.dropna()
#np.mean(MAPEs)
#consumption_df.iloc[100]
predicted_values=pd.DataFrame(predicted_values)
predicted_values=predicted_values.drop(predicted_values.index[0])
for column in predicted_values.columns: 
     pred_list=pred_list+predicted_values[column].tolist()
consumption_df['HW_predictions']=pred_list
consumption_df['HW_predictions']=consumption_df['HW_predictions'].shift(48)
eval_function("HW_foreach_hour",consumption_df[consumption_df.index>47].dropna()["Consumption"],consumption_df[consumption_df.index>47].dropna()['HW_predictions'])


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:215: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:215: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:215: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/u

RMSE:  3823.2939187063193
MAE:  2705.8267471272175
MAPE:  8.531507964754685


('HW_foreach_hour', 3823.2939187063193, 2705.8267471272175, 8.531507964754685)

In [0]:
consumption_df[consumption_df.index>46]["HW_predictions"]

#HW Including Special Days by Regression Coefficients


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

In [139]:
special_days_path="drive/My Drive/ozel_gunler/*"
special_day_df=pd.DataFrame()
exception=[]
for i in tqdm(glob.glob(special_days_path)):  
    a=pd.read_csv(i)
    try:
        a["special_day"]=np.full(len(a),a.columns[0])
        a.columns=["date","special_day"]
        special_day_df=pd.concat([special_day_df, a])
    except ValueError:
        exception.append(i)
        pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [0]:
special_day_df["is"]=np.ones(len(special_day_df))
special_day_df=pd.pivot_table(special_day_df,index="date",columns="special_day").fillna(0).astype(int)
special_day_df.columns=['_'.join(i) for i in special_day_df.columns]
#special_day_df
special_day_df=special_day_df.reset_index()

In [141]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
special_day_df["date"]=special_day_df["date"].apply(lambda x:'.'.join(list(reversed(x.split("-")))))
#special_day_df["date"].dtype
cons_lagged=pd.merge(cons_lagged,special_day_df,left_on="Date",right_on="date",how="left").drop(columns="date").fillna(0)
#from sklearn.preprocessing import OneHotEncoder
#from scipy.sparse import hstack
#cat_features=["Hour","Day","Month","weekday"]
#linear_enc=OneHotEncoder()
#one_hot_encoded=linear_enc.fit_transform(cons_lagged[cat_features]).toarray()
#normal_feats=cons_lagged.columns.difference(cat_features).tolist()
#cons_lagged=np.hstack([cons_lagged[normal_feats],one_hot_encoded])
#cons_lagged=pd.DataFrame(cons_lagged,columns=normal_feats+np.concatenate([[cat_features[i]+"_"+str(j) for j in cats_]for i,cats_ in enumerate(linear_enc.categories_)]).tolist())
cons_lagged=cons_lagged.dropna()
cons_lagged.iloc[0:50]

,Date,Hour,Consumption,Datetime,HW_predictions,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,0,29590.88,2015-12-31 00:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,31.12.2015,1,27785.43,2015-12-31 01:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,31.12.2015,2,26516.53,2015-12-31 02:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,31.12.2015,3,26091.62,2015-12-31 03:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,31.12.2015,4,25871.54,2015-12-31 04:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,31.12.2015,5,26053.84,2015-12-31 05:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,31.12.2015,6,26889.99,2015-12-31 06:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,31.12.2015,7,28103.02,2015-12-31 07:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,31.12.2015,8,32321.87,2015-12-31 08:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,31.12.2015,9,35759.14,2015-12-31 09:00:00,0.000000,0.00,0.0,31,12,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
cons_lagged_unmodif=cons_lagged.copy()
cons_lagged=cons_lagged.iloc[48:,]
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00",format="%Y-%m-%d %H:%M")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00",format="%Y-%m-%d %H:%M")]
from sklearn.linear_model import LinearRegression
cons_lagged=cons_lagged.dropna()
cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values
model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
sp_days_index=[i for i,col in enumerate(train_df[cols].columns) if col.startswith("is_")]
cons_lagged=pd.merge(consumption_df,special_day_df,left_on="Date",right_on="date",how="left").drop(columns="date").fillna(0)


In [0]:
special_days_effect_dict=dict(zip(train_df[cols].columns[sp_days_index].tolist(),model.coef_[0][sp_days_index].tolist()))
for key_ in special_days_effect_dict.keys():
    cons_lagged[key_]=cons_lagged[key_]*special_days_effect_dict[key_]
#cons_lagged["Consumption"]=cons_lagged["Consumption"]-cons_lagged[list(special_days_effect_dict.keys())].sum(axis=1)
special_clmn=[i for i,col in enumerate(cons_lagged.columns) if col.startswith("is_")]
#cons_lagged.ix[:,107]
#sp_days_index
#cons_lagged.isnull().sum()

In [169]:
#cons_lagged['HW_predictions']=ExponentialSmoothing(consumption_df["Consumption"], seasonal_periods=24, trend='add', seasonal='add').fit(use_boxcox=True).fittedvalues
cons_lagged_exp=cons_lagged.copy()
#cons_lagged_exp['modified_pred_by_regression_coeff']=cons_lagged_exp['HW_predictions']+(cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index]).sum(axis=1)
cons_lagged_exp['modified_pred_by_regression_coeff']=cons_lagged_exp['HW_predictions']+(cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index]).mean(axis=1)
cons_lagged_exp_test=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]
cons_lagged_exp_train=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
exp_eval_df.append(eval_function("HW_modified_by_regression",cons_lagged_exp_test["Consumption"],cons_lagged_exp_test['modified_pred_by_regression_coeff']))
#cons_lagged_exp.loc[cons_lagged_exp['is_arife']==1]
#cons_lagged_exp[["Consumption","modified_pred_by_regression_coeff"]]
#cons_lagged['modified_pred_by_regression_coeff'].isnull().sum()#,cons_lagged['modified_pred_by_regression_coeff']

RMSE:  3997.137412515099
MAE:  2869.697620049824
MAPE:  8.896344358209433


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


(cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index])["is_cocukbayrami"].iloc[[ 2712,  2713,  2714,  2715,  2716,  2717,  2718,  2719,  2720,
         2721,  2722,  2723,  2724,  2725,  2726,  2727,  2728,  2729,
         2730,  2731,  2732,  2733,  2734,  2735, 11472, 11473, 11474,
        11475, 11476, 11477, 11478, 11479, 11480, 11481, 11482, 11483,
        11484, 11485, 11486, 11487, 11488, 11489, 11490, 11491, 11492,
        11493, 11494, 11495, 20232, 20233, 20234, 20235, 20236, 20237,
        20238, 20239, 20240, 20241, 20242, 20243, 20244, 20245, 20246,
        20247, 20248, 20249, 20250, 20251, 20252, 20253, 20254, 20255,
        28992, 28993, 28994, 28995, 28996, 28997, 28998, 28999, 29000,
        29001, 29002, 29003, 29004, 29005, 29006, 29007, 29008, 29009,
        29010, 29011, 29012, 29013, 29014, 29015]]

In [0]:
#np.count_nonzero((cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index]).sum(axis=1))
cons_lagged_exp_test[["Consumption",'modified_pred_by_regression_coeff']]

#HW Including Special Days as Error Factor by the Previous Years 

In [149]:
cons_lagged_exp["HW_percent_errors"]=(cons_lagged_exp['HW_predictions']-cons_lagged_exp['Consumption'])/cons_lagged_exp['Consumption']
cons_lagged_exp_test=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]
cons_lagged_exp_train=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
cons_lagged_exp_train=cons_lagged_exp_train.drop(cons_lagged_exp_train.index[0:47])
#model=LinearRegression()
#model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
#cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values
#cons_lagged_exp["HW_error"]
#Ex=ExponentialSmoothing(train_df["Consumption"], seasonal_periods=24, trend='add', seasonal='add').fit(use_boxcox=True)
#scaled_holiday_errors=(Ex.fittedvalues-train_df.loc[consumption_df["Datetime"]<["Consumption"])/train_df["Consumption"]
model_by_error=LinearRegression(fit_intercept=False)
model_by_error.fit(cons_lagged_exp_train.ix[:,special_clmn].values,cons_lagged_exp_train["HW_percent_errors"].values.reshape(-1, 1))
cons_lagged_exp_test['modified_by_prev_error']=cons_lagged_exp_test['HW_predictions']*(1-(cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0]).max(axis=1))
exp_eval_df.append(eval_function("HW_modified_by_error",cons_lagged_exp_test["Consumption"],cons_lagged_exp_test['modified_by_prev_error']))
cons_lagged_exp_test["HW_percent_errors"]
#(cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0]).max(axis=1)
#cons_lagged_exp_test=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]model_by_error.coef_
#cons_lagged_exp.ix[:,special_clmn]
cons_lagged_exp_test.loc[cons_lagged_exp_test['is_arife']==1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]

RMSE:  1.6248018528982778e+16
MAE:  9351271206946546.0
MAPE:  26598896957442.46


,Date,Hour,Consumption,Datetime,HW_predictions,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami,modified_pred_by_regression_coeff,HW_percent_errors,modified_by_prev_error
29976,03.06.2019,00:00,29170.08,2019-06-03 00:00:00,35236.036807,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,30960.217489,0.207951,-1.673537e+15
29977,03.06.2019,01:00,27813.43,2019-06-03 01:00:00,33884.531110,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,29608.711792,0.218279,-1.609347e+15
29978,03.06.2019,02:00,27022.75,2019-06-03 02:00:00,32635.826105,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,28360.006787,0.207717,-1.550040e+15
29979,03.06.2019,03:00,25857.15,2019-06-03 03:00:00,31360.982515,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,27085.163197,0.212855,-1.489491e+15
29980,03.06.2019,04:00,24259.94,2019-06-03 04:00:00,30259.098487,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,25983.279169,0.247287,-1.437157e+15
29981,03.06.2019,05:00,22296.62,2019-06-03 05:00:00,28383.200597,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,24107.381279,0.272982,-1.348061e+15
29982,03.06.2019,06:00,21397.26,2019-06-03 06:00:00,27529.218608,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,23253.399291,0.286577,-1.307501e+15
29983,03.06.2019,07:00,21561.23,2019-06-03 07:00:00,27996.006018,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,23720.186700,0.298442,-1.329671e+15
29984,03.06.2019,08:00,22652.43,2019-06-03 08:00:00,31501.470157,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,27225.650839,0.390644,-1.496163e+15
29985,03.06.2019,09:00,24068.36,2019-06-03 09:00:00,34161.399008,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,29885.579690,0.419349,-1.622497e+15


In [0]:
model_by_error.coef_[0]

In [0]:
error_eff=[]

for co in cons_lagged_exp_train.ix[:,special_clmn].columns:
  
     error_eff+=[(cons_lagged_exp_train.loc[cons_lagged_exp_train[co]==1]["HW_percent_errors"]/cons_lagged_exp_train.loc[cons_lagged_exp_train[co]==1].ix[:,special_clmn].sum(axis=1)).mean(axis=0)]
error_eff
cons_lagged_exp_test['modified_by_prev_error2']=cons_lagged_exp_test['HW_predictions']*(1-(cons_lagged_exp_test.ix[:,special_clmn]*error_eff).max(axis=1))
exp_eval_df.append(eval_function("HW_modified_by_error2",cons_lagged_exp_test["Consumption"],cons_lagged_exp_test['modified_by_prev_error2']))


In [147]:
cons_lagged_exp_train.loc[cons_lagged_exp_train['is_arife']==1]["HW_percent_errors"]

4440     0.143029
4441     0.154152
4442     0.152180
4443     0.156593
4444     0.192367
           ...   
23107    0.223950
23108    0.206804
23109    0.204668
23110    0.209900
23111    0.208384
Name: HW_percent_errors, Length: 144, dtype: float64

In [0]:
a=0
for co in cons_lagged_exp_train.ix[:,special_clmn].columns[0]
   a+=1

SyntaxError: ignored

ERROR! Session/line number was not unique in database. History logging moved to new session 89


In [0]:
cons_lagged_exp_train.ix[:,special_clmn].columns()

ERROR! Session/line number was not unique in database. History logging moved to new session 65


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


TypeError: ignored

In [0]:
np.nonzero((cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0])['is_arife'])[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


array([2184, 3816])

In [0]:
#cons_lagged_exp_test['HW_predictions']*
#cons_lagged_exp_test[np.nonzero((cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0])['is_arife'])]

cons_lagged_exp_test.iloc[2184]

Date                                          03.06.2019
Hour                                               00:00
Consumption                                      29170.1
Datetime                             2019-06-03 00:00:00
is_annelergunu                                         0
is_arife                                         3578.49
is_babalargunu                                         0
is_cocukbayrami                                        0
is_cumhuriyet                                          0
is_elections                                           0
is_examination                                         0
is_genclikbayrami                                      0
is_holiday                                      -3360.78
is_iscibayrami                                        -0
is_kadinlargunu                                       -0
is_kandil                                              0
is_kurbanbayrami                                      -0
is_kurbanbayramilkgun          

In [0]:
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged_exp_test["Date"]=="11.08.2019"]["Consumption"],cons_lagged_exp_test.loc[cons_lagged_exp_test['Date']=="11.08.2019"]['modified_by_prev_error'])
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged_exp_test["Date"]=="29.11.2019"]["Consumption"],cons_lagged_exp_test.loc[cons_lagged_exp_test['Date']=="29.11.2019"]['modified_by_prev_error'])

RMSE:  1635.3531257507652
MAE:  1199.3832340265842
MAPE:  4.686318505811885
RMSE:  539.4578995073159
MAE:  423.815928634306
MAPE:  1.1585038298968189


('HW_modified_by_error',
 539.4578995073159,
 423.815928634306,
 1.1585038298968189)

In [0]:
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged["Date"]=="11.08.2019"]["Consumption"],cons_lagged.loc[cons_lagged['Date']=="11.08.2019"]['HW_predictions'])
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged["Date"]=="29.11.2019"]["Consumption"],cons_lagged.loc[cons_lagged['Date']=="29.11.2019"]['HW_predictions'])

RMSE:  1636.2998414239462
MAE:  1205.3475423066493
MAPE:  4.706172366712435
RMSE:  539.4318065258644
MAE:  423.36845960858153
MAPE:  1.1570989172735997


('HW_modified_by_error',
 539.4318065258644,
 423.36845960858153,
 1.1570989172735997)

#HW by Interpolated Data and Naive Approach for Special Days(mantıklı olmayabilir)